In [1]:
import sys, pathlib, importlib, pennylane as qml
PROJECT_ROOT = pathlib.Path.cwd().resolve().parent.parent
SRC = PROJECT_ROOT / "src"
if str(SRC) not in sys.path:
    sys.path.append(str(SRC))

import circuit.quantum_galton_board as qgb
importlib.reload(qgb)

<module 'circuit.quantum_galton_board' from 'C:\\Users\\13174\\Documents\\GaltonBoard\\src\\circuit\\quantum_galton_board.py'>

In [2]:

levels = 8
board_core = qgb.build_galton_circuit(
    levels    = levels,
    num_shots = None,        # analytic
    bias      = 0.3,
    coherence = True,        # no resets → no measurements
)
_ = board_core.construct([], {})          # build tape once
               # should print 15 not sure why it's giving 36 wires


In [ ]:
#QAOA layer on wires 8-15 to cut the circuit smaller

data_wires = list(range(levels, 2*levels))         # 8..15
def maxcut_cost(wires):
    H = qml.Hamiltonian([], [])
    for i in range(len(wires)):
        for j in range(i+1, len(wires)):
            H += 0.5*(qml.Identity(0) - qml.PauliZ(wires[i]) @ qml.PauliZ(wires[j]))
    return H
cost_H = maxcut_cost(data_wires)

gamma = qml.numpy.array(0.8, requires_grad=True)
beta  = qml.numpy.array(0.4, requires_grad=True)
def qaoa_layer():
    qml.templates.ApproxTimeEvolution(cost_H, gamma, 1)
    for w in data_wires:
        qml.RX(2*beta, wires=w)

dev_full = qml.device("lightning.qubit", wires=2*levels)

@qml.qnode(dev_full)
def full_circuit():
    board_core()
    qaoa_layer()
    return qml.expval(cost_H)

print("Full-circuit ⟨H⟩ =", full_circuit())

WireError: Cannot run circuit(s) on lightning.qubit as they contain wires not found on the device: {16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36}

In [7]:
#so now we build the circuit with da qaoa layer

dev_full = qml.device("lightning.qubit", wires=2*levels)

@qml.qnode(dev_full)
def full_circuit():
    board_core()            # 16-wire coherent board
    qaoa_layer()            # single QAOA block
    return qml.expval(cost_H)

print("Full-circuit ⟨H⟩:", full_circuit())

WireError: Cannot run circuit(s) on lightning.qubit as they contain wires not found on the device: {16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36}